In [3]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import json

from datetime import datetime, timedelta

In [153]:
# Read bloomberg data
raw_bloomberg_df = pd.read_csv('usd_idr_data.csv', header=1)

# Rename header
raw_bloomberg_df = raw_bloomberg_df.rename(columns={'Unnamed: 0': 'Datetime'})

raw_bloomberg_df

,Datetime,PX_OPEN,PX_LAST,PX_HIGH,PX_LOW,BID,ASK,PX_MID,high,low
0,1991-11-05,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
1,1991-11-06,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
2,1991-11-07,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
3,1991-11-08,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
4,1991-11-11,1977.0,1977.0,1977.0,1977.0,NaN,NaN,1977.0,1977.0,1977.0
...,...,...,...,...,...,...,...,...,...,...
8379,2024-10-22,15534.0,15560.0,15572.0,15534.0,15555.0,15565.0,15560.0,15572.0,15534.0
8380,2024-10-23,15590.0,15620.0,15633.0,15590.0,15615.0,15625.0,15620.0,15633.0,15590.0
8381,2024-10-24,15640.0,15584.0,15644.0,15580.0,15583.0,15585.0,15584.0,15644.0,15580.0
8382,2024-10-25,15622.0,15640.0,15643.0,15603.0,15635.0,15645.0,15640.0,15643.0,15603.0


In [6]:
# Read bloomberg data
raw_bloomberg_df = pd.read_csv('btc_daily.csv', header=0)

# Convert opentime and closetime from milliseconds to datetime format YYYY-MM-DD
raw_bloomberg_df['opentime'] = pd.to_datetime(raw_bloomberg_df['opentime'], unit='ms').dt.strftime('%Y-%m-%d')
raw_bloomberg_df['closetime'] = pd.to_datetime(raw_bloomberg_df['closetime'], unit='ms').dt.strftime('%Y-%m-%d')

raw_bloomberg_df

,coin,opentime,openprice,highprice,lowprice,closeprice,volume,closetime,quotevolume,trades,taker_buy_volume,taker_buy_quote,unused
0,BTC,2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291,2019-09-08,3.209628e+07,3754,0.039,3.933563e+02,0
1,BTC,2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373,2019-09-09,1.524472e+08,11296,5452.451,5.619740e+07,0
2,BTC,2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955,2019-09-10,9.271765e+07,10089,6395.462,6.552636e+07,0
3,BTC,2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922,2019-09-11,1.097862e+08,13811,7724.442,7.784116e+07,0
4,BTC,2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634,2019-09-12,1.594942e+08,20060,10117.798,1.031607e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,BTC,2024-10-28,67992.70,70331.00,67584.20,69940.00,256128.622,2024-10-28,1.764992e+10,3334954,135854.890,9.364715e+09,0
1878,BTC,2024-10-29,69939.90,73660.00,69733.00,72739.50,382502.060,2024-10-29,2.750227e+10,4796294,199025.299,1.431096e+10,0
1879,BTC,2024-10-30,72739.50,72984.90,71444.20,72349.00,227108.808,2024-10-30,1.639468e+10,3249471,113211.086,8.173511e+09,0
1880,BTC,2024-10-31,72349.00,72720.30,69705.00,70321.90,256485.328,2024-10-31,1.826086e+10,3578458,122732.500,8.739091e+09,0


In [5]:
raw_bloomberg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   coin              1882 non-null   object 
 1   opentime          1882 non-null   int64  
 2   openprice         1882 non-null   float64
 3   highprice         1882 non-null   float64
 4   lowprice          1882 non-null   float64
 5   closeprice        1882 non-null   float64
 6   volume            1882 non-null   float64
 7   closetime         1882 non-null   int64  
 8   quotevolume       1882 non-null   float64
 9   trades            1882 non-null   int64  
 10  taker_buy_volume  1882 non-null   float64
 11  taker_buy_quote   1882 non-null   float64
 12  unused            1882 non-null   int64  
dtypes: float64(8), int64(4), object(1)
memory usage: 191.3+ KB


In [75]:
# # Find rows where PX_OPEN, PX_LAST, PX_HIGH, and PX_LOW have the same value
# same_value_rows = raw_bloomberg_df[
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_LAST']) &
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_HIGH']) &
#     (raw_bloomberg_df['PX_OPEN'] == raw_bloomberg_df['PX_LOW'])
# ]

# # Display the rows with the same values
# same_value_rows

In [154]:
# Make sure that 'Datetime; is in the correct format
raw_bloomberg_df['Datetime'] = pd.to_datetime(raw_bloomberg_df['Datetime'])

# Filter the data from year 1995 only because the years before has the same value for the OHLC
df = raw_bloomberg_df[raw_bloomberg_df['Datetime'].dt.year >= 1994]

# Take necessary columns
df = df[['Datetime','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST']]

# Rename the columns
df.columns = ['date','open', 'high', 'low', 'close']

df

,date,open,high,low,close
564,1994-01-03,2109.0,2109.0,2109.0,2109.0
565,1994-01-04,2106.0,2106.0,2106.0,2106.0
566,1994-01-05,2105.0,2105.0,2105.0,2105.0
567,1994-01-06,2103.0,2103.0,2103.0,2103.0
568,1994-01-07,2102.0,2102.0,2102.0,2102.0
...,...,...,...,...,...
8379,2024-10-22,15534.0,15572.0,15534.0,15560.0
8380,2024-10-23,15590.0,15633.0,15590.0,15620.0
8381,2024-10-24,15640.0,15644.0,15580.0,15584.0
8382,2024-10-25,15622.0,15643.0,15603.0,15640.0


In [7]:
raw_bloomberg_df.columns

Index(['coin', 'opentime', 'openprice', 'highprice', 'lowprice', 'closeprice',
       'volume', 'closetime', 'quotevolume', 'trades', 'taker_buy_volume',
       'taker_buy_quote', 'unused'],
      dtype='object')

In [8]:
# Take necessary columns
df = raw_bloomberg_df[['opentime','openprice','highprice','lowprice','closeprice']]

# Rename the columns
df.columns = ['date','open', 'high', 'low', 'close']

df

,date,open,high,low,close
0,2019-09-08,10000.00,10412.65,10000.00,10391.63
1,2019-09-09,10316.62,10475.54,10077.22,10307.00
2,2019-09-10,10307.00,10382.97,9940.87,10102.02
3,2019-09-11,10094.27,10293.11,9884.31,10159.55
4,2019-09-12,10163.06,10450.13,10042.12,10415.13
...,...,...,...,...,...
1877,2024-10-28,67992.70,70331.00,67584.20,69940.00
1878,2024-10-29,69939.90,73660.00,69733.00,72739.50
1879,2024-10-30,72739.50,72984.90,71444.20,72349.00
1880,2024-10-31,72349.00,72720.30,69705.00,70321.90


In [9]:
# Display rows with any null values in the 'open', 'high', 'low', or 'close' columns
missing_values_df = df[df[['open', 'high', 'low', 'close']].isnull().any(axis=1)]
missing_values_df

,date,open,high,low,close


In [145]:
# # only the data from 2008-2011 has this issue

# # Drop rows where both 'open' and 'high' are null
# df = df.dropna(subset=['open', 'high'], how='all')

# # Step 2: Fill null values in 'low' and 'close' with values from 'open'
# df['low'] = df['low'].fillna(df['open']) # open == high
# df['close'] = df['close'].fillna(df['open']) # open == high

# # Display the modified DataFrame
# df

In [10]:
# Check for any missing values
print(df.isnull().sum())

date     0
open     0
high     0
low      0
close    0
dtype: int64


In [ ]:
# Index(['date', 'open', 'high', 'low', 'close', 'gap_day', 'next_close',
#        'open_change', 'high_change', 'low_change', 'close_change',
#        'ma_close_5', 'ma_close_10', 'prev_open', 'prev_high', 'prev_low',
#        'prev_close', 'daily_range', 'gap_change', 'close_ma5_ratio',
#        'close_ma10_ratio'],
#       dtype='object')

In [11]:
# Ensure 'date' is consistently used as a datetime format
df['date'] = pd.to_datetime(df['date'])

# Features Engineering

# Create the next_close column as the target variable to be predicted
df['next_close'] = df['close'].shift(-1)

# Count the next day gap
df['gap_day'] = df['date'].shift(-1) - df['date']
# Convert gap_day to just the number of days and make it an integer
df['gap_day'] = df['gap_day'].dt.days.fillna(0).astype(int)

# Calculate the changes from the previous row for OHLC
df['open_change'] = df['open'].diff()
df['high_change'] = df['high'].diff()
df['low_change'] = df['low'].diff()
df['close_change'] = df['close'].diff()

# Function to calculate moving average over a custom date range
def custom_moving_average(df, close_col, date_col, days):
    averages = []
    for index, row in df.iterrows():
        current_date = row[date_col]
        # Get the date range
        start_date = current_date - pd.Timedelta(days=days)
        # Filter the DataFrame for the desired date range
        mask = (df[date_col] >= start_date) & (df[date_col] <= current_date)
        # Calculate the mean for the filtered values
        avg = df.loc[mask, close_col].mean()
        averages.append(avg)
    return averages

# Calculate the custom moving averages
df['ma_close_5'] = custom_moving_average(df, 'close', 'date', days=4)  # 4 previous days + today
df['ma_close_10'] = custom_moving_average(df, 'close', 'date', days=9)  # 9 previous days + today

# Capture previous values
df['prev_open'] = df['open'].shift(1)
df['prev_high'] = df['high'].shift(1)
df['prev_low'] = df['low'].shift(1)
df['prev_close'] = df['close'].shift(1)

# Add daily range (high-low)
df['daily_range'] = df['high'] - df['low']

# Add gap change (today's open - previous close)
df['gap_change'] = df['open'] - df['prev_close']

# Add close-to-moving average ratios
df['close_ma5_ratio'] = df['close'] / df['ma_close_5']
df['close_ma10_ratio'] = df['close'] / df['ma_close_10']

# Drop columns that are not needed for prediction
df = df[['date', 'open', 'high', 'low', 'close', 'gap_day', 
         'next_close', 'open_change', 'high_change', 
         'low_change', 'close_change', 'ma_close_5', 
         'ma_close_10', 'prev_open', 'prev_high', 
         'prev_low', 'prev_close', 'daily_range', 'gap_change',
         'close_ma5_ratio', 'close_ma10_ratio']]

# Filter the DataFrame to include only rows from the year 2012 onwards
df = df[df['date'] >= '2012-01-01']

# Dropping last row
df = df.drop(df.index[-1])

df

/tmp/ipykernel_1076460/2002822642.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/tmp/ipykernel_1076460/2002822642.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_close'] = df['close'].shift(-1)
/tmp/ipykernel_1076460/2002822642.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,date,open,high,low,close,gap_day,next_close,open_change,high_change,low_change,...,ma_close_5,ma_close_10,prev_open,prev_high,prev_low,prev_close,daily_range,gap_change,close_ma5_ratio,close_ma10_ratio
0,2019-09-08,10000.00,10412.65,10000.00,10391.63,1,10307.00,NaN,NaN,NaN,...,10391.630000,10391.630000,NaN,NaN,NaN,NaN,412.65,NaN,1.000000,1.000000
1,2019-09-09,10316.62,10475.54,10077.22,10307.00,1,10102.02,316.62,62.89,77.22,...,10349.315000,10349.315000,10000.00,10412.65,10000.00,10391.63,398.32,-75.01,0.995911,0.995911
2,2019-09-10,10307.00,10382.97,9940.87,10102.02,1,10159.55,-9.62,-92.57,-136.35,...,10266.883333,10266.883333,10316.62,10475.54,10077.22,10307.00,442.10,0.00,0.983942,0.983942
3,2019-09-11,10094.27,10293.11,9884.31,10159.55,1,10415.13,-212.73,-89.86,-56.56,...,10240.050000,10240.050000,10307.00,10382.97,9940.87,10102.02,408.80,-7.75,0.992139,0.992139
4,2019-09-12,10163.06,10450.13,10042.12,10415.13,1,10341.34,68.79,157.02,157.81,...,10275.066000,10275.066000,10094.27,10293.11,9884.31,10159.55,408.01,3.51,1.013631,1.013631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,2024-10-27,67071.90,68306.10,66909.40,67992.60,1,69940.00,409.20,856.10,495.30,...,67309.640000,67710.840000,66662.70,67450.00,66414.10,67071.80,1396.70,0.10,1.010147,1.004161
1877,2024-10-28,67992.70,70331.00,67584.20,69940.00,1,72739.50,920.80,2024.90,674.80,...,67968.200000,67864.060000,67071.90,68306.10,66909.40,67992.60,2746.80,0.10,1.029011,1.030590
1878,2024-10-29,69939.90,73660.00,69733.00,72739.50,1,72349.00,1947.20,3329.00,2148.80,...,68881.340000,68302.300000,67992.70,70331.00,67584.20,69940.00,3927.00,-0.10,1.056012,1.064964
1879,2024-10-30,72739.50,72984.90,71444.20,72349.00,1,70321.90,2799.60,-675.10,1711.20,...,70018.580000,68635.190000,69939.90,73660.00,69733.00,72739.50,1540.70,0.00,1.033283,1.054109


In [12]:
# Dropping first row
df = df.drop(df.index[0])

df

,date,open,high,low,close,gap_day,next_close,open_change,high_change,low_change,...,ma_close_5,ma_close_10,prev_open,prev_high,prev_low,prev_close,daily_range,gap_change,close_ma5_ratio,close_ma10_ratio
1,2019-09-09,10316.62,10475.54,10077.22,10307.00,1,10102.02,316.62,62.89,77.22,...,10349.315000,10349.315000,10000.00,10412.65,10000.00,10391.63,398.32,-75.01,0.995911,0.995911
2,2019-09-10,10307.00,10382.97,9940.87,10102.02,1,10159.55,-9.62,-92.57,-136.35,...,10266.883333,10266.883333,10316.62,10475.54,10077.22,10307.00,442.10,0.00,0.983942,0.983942
3,2019-09-11,10094.27,10293.11,9884.31,10159.55,1,10415.13,-212.73,-89.86,-56.56,...,10240.050000,10240.050000,10307.00,10382.97,9940.87,10102.02,408.80,-7.75,0.992139,0.992139
4,2019-09-12,10163.06,10450.13,10042.12,10415.13,1,10341.34,68.79,157.02,157.81,...,10275.066000,10275.066000,10094.27,10293.11,9884.31,10159.55,408.01,3.51,1.013631,1.013631
5,2019-09-13,10414.96,10440.55,10153.51,10341.34,1,10332.25,251.90,-9.58,111.39,...,10265.008000,10286.111667,10163.06,10450.13,10042.12,10415.13,287.04,-0.17,1.007436,1.005369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,2024-10-27,67071.90,68306.10,66909.40,67992.60,1,69940.00,409.20,856.10,495.30,...,67309.640000,67710.840000,66662.70,67450.00,66414.10,67071.80,1396.70,0.10,1.010147,1.004161
1877,2024-10-28,67992.70,70331.00,67584.20,69940.00,1,72739.50,920.80,2024.90,674.80,...,67968.200000,67864.060000,67071.90,68306.10,66909.40,67992.60,2746.80,0.10,1.029011,1.030590
1878,2024-10-29,69939.90,73660.00,69733.00,72739.50,1,72349.00,1947.20,3329.00,2148.80,...,68881.340000,68302.300000,67992.70,70331.00,67584.20,69940.00,3927.00,-0.10,1.056012,1.064964
1879,2024-10-30,72739.50,72984.90,71444.20,72349.00,1,70321.90,2799.60,-675.10,1711.20,...,70018.580000,68635.190000,69939.90,73660.00,69733.00,72739.50,1540.70,0.00,1.033283,1.054109


In [14]:
# Drop the last 6 rows from dfr
df = df.iloc[:-6]

# Display the updated DataFrame
df

,date,open,high,low,close,gap_day,next_close,open_change,high_change,low_change,...,ma_close_5,ma_close_10,prev_open,prev_high,prev_low,prev_close,daily_range,gap_change,close_ma5_ratio,close_ma10_ratio
1,2019-09-09,10316.62,10475.54,10077.22,10307.00,1,10102.02,316.62,62.89,77.22,...,10349.315000,10349.315000,10000.00,10412.65,10000.00,10391.63,398.32,-75.01,0.995911,0.995911
2,2019-09-10,10307.00,10382.97,9940.87,10102.02,1,10159.55,-9.62,-92.57,-136.35,...,10266.883333,10266.883333,10316.62,10475.54,10077.22,10307.00,442.10,0.00,0.983942,0.983942
3,2019-09-11,10094.27,10293.11,9884.31,10159.55,1,10415.13,-212.73,-89.86,-56.56,...,10240.050000,10240.050000,10307.00,10382.97,9940.87,10102.02,408.80,-7.75,0.992139,0.992139
4,2019-09-12,10163.06,10450.13,10042.12,10415.13,1,10341.34,68.79,157.02,157.81,...,10275.066000,10275.066000,10094.27,10293.11,9884.31,10159.55,408.01,3.51,1.013631,1.013631
5,2019-09-13,10414.96,10440.55,10153.51,10341.34,1,10332.25,251.90,-9.58,111.39,...,10265.008000,10286.111667,10163.06,10450.13,10042.12,10415.13,287.04,-0.17,1.007436,1.005369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,2024-10-21,69020.20,69566.00,66865.90,67368.80,1,67406.40,663.10,140.20,-1203.90,...,68111.060000,66727.210000,68357.10,69425.80,68069.80,69020.10,2700.10,0.10,0.989102,1.009615
1871,2024-10-22,67368.70,67825.00,66536.50,67406.40,1,66647.20,-1651.50,-1741.00,-329.40,...,68112.040000,67150.250000,69020.20,69566.00,66865.90,67368.80,1288.50,-0.10,0.989640,1.003815
1872,2024-10-23,67406.50,67449.00,65222.30,66647.20,1,68173.80,37.80,-376.00,-1314.20,...,67759.920000,67530.990000,67368.70,67825.00,66536.50,67406.40,2226.70,0.10,0.983578,0.986913
1873,2024-10-24,66647.30,68842.50,66482.00,68173.80,1,66662.80,-759.20,1393.50,1259.70,...,67723.260000,67741.880000,67406.50,67449.00,65222.30,66647.20,2360.50,0.10,1.006653,1.006376


In [15]:
# Count the frequency of each unique value in the 'gap_day' column
gap_day_frequency = df['gap_day'].value_counts()

# Display the frequency counts
print(gap_day_frequency)

gap_day
1    1874
Name: count, dtype: int64


In [16]:
# Specify the filename
filename = 'binance_btc_2019_2024_processed.csv'

# Write the DataFrame to a CSV file
df.to_csv(filename, index=False)

print(f'DataFrame written to {filename}')

DataFrame written to binance_btc_2019_2024_processed.csv
